In [74]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno
import psycopg2
import getpass

In [75]:
# Request user for password to access database
db_password = getpass.getpass(prompt='DB Password ')

# Create SQL engine with connection parameters to connect to AWS RDS Postgres instance
engine = psycopg2.connect(
    database="Stores",
    user="postgres",
    password=db_password,
    host="stores-db.ck6vux0ulqkn.us-east-2.rds.amazonaws.com",
    port='5432'
)

DB Password ········


In [76]:
# Show sample of the imported data
train = pd.read_sql('SELECT * FROM store_table', engine)
train.head(1)

,store_id,store_area,items_available,daily_customer_count,store_sales
0,1,1659,1961,530,66490


In [77]:
train

,store_id,store_area,items_available,daily_customer_count,store_sales
0,1,1659,1961,530,66490
1,2,1461,1752,210,39820
2,3,1340,1609,720,54010
3,4,1451,1748,620,53730
4,5,1770,2111,450,46620
...,...,...,...,...,...
891,892,1582,1910,1080,66390
892,893,1387,1663,850,82080
893,894,1200,1436,1060,76440
894,895,1299,1560,770,96610


In [78]:
train.drop(train.columns[[0]], axis = 1, inplace = True)

In [79]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   store_area            896 non-null    int64
 1   items_available       896 non-null    int64
 2   daily_customer_count  896 non-null    int64
 3   store_sales           896 non-null    int64
dtypes: int64(4)
memory usage: 28.1 KB


In [80]:
train.describe()

,store_area,items_available,daily_customer_count,store_sales
count,896.000000,896.000000,896.000000,896.000000
mean,1485.409598,1782.035714,786.350446,59351.305804
std,250.237011,299.872053,265.389281,17190.741895
min,775.000000,932.000000,10.000000,14920.000000
25%,1316.750000,1575.500000,600.000000,46530.000000
50%,1477.000000,1773.500000,780.000000,58605.000000
75%,1653.500000,1982.750000,970.000000,71872.500000
max,2229.000000,2667.000000,1560.000000,116320.000000


In [81]:
train.isnull().sum()

store_area              0
items_available         0
daily_customer_count    0
store_sales             0
dtype: int64

In [82]:
train.duplicated().sum()

0

In [83]:
for column_name in train.columns:
    unique_values = len(train[column_name].unique())
    print("Feature '{column_name}' has '{unique_values}' unique values".format(column_name = column_name,
                                                                                         unique_values=unique_values))

Feature 'store_area' has '583' unique values
Feature 'items_available' has '616' unique values
Feature 'daily_customer_count' has '130' unique values
Feature 'store_sales' has '816' unique values


In [84]:
# train=train.drop('daily_customer_count',axis=1)

In [85]:
train.head()

,store_area,items_available,daily_customer_count,store_sales
0,1659,1961,530,66490
1,1461,1752,210,39820
2,1340,1609,720,54010
3,1451,1748,620,53730
4,1770,2111,450,46620


In [86]:
from sklearn.neighbors import LocalOutlierFactor

In [87]:
clf = LocalOutlierFactor(n_neighbors=5, contamination='auto')
y_pred = clf.fit_predict(train) 

In [88]:
train['Out']=y_pred

In [89]:
Out=train[train['Out']!=1]

In [90]:
Out.shape

(23, 5)

In [91]:
train2=train[train['Out']==1]

In [92]:
train2

,store_area,items_available,daily_customer_count,store_sales,Out
0,1659,1961,530,66490,1
1,1461,1752,210,39820,1
2,1340,1609,720,54010,1
3,1451,1748,620,53730,1
4,1770,2111,450,46620,1
...,...,...,...,...,...
891,1582,1910,1080,66390,1
892,1387,1663,850,82080,1
893,1200,1436,1060,76440,1
894,1299,1560,770,96610,1


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [94]:
y=train2['store_sales']
X=train2.drop(['store_sales','Out'],axis=1)

In [95]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.4,random_state=1)

In [96]:
models = [LinearRegression(),RandomForestRegressor(),xgb.XGBRegressor()]
scores = dict()

for m in models:
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    s=m.score(X_train,y_train)

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100}')
    print(f'Score: {s}')
    print('-'*30, '\n')
    

model: LinearRegression()
MAPE: 27.433119221793206
Score: 0.01746727631820244
------------------------------ 

model: RandomForestRegressor()
MAPE: 29.202681386560354
Score: 0.8460213941644855
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 32.22029130225478
Score: 0.9830497602389252
------------------------------ 



In [97]:
#LogisticRegression
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [98]:
regressor = linear_model.LogisticRegression(solver='lbfgs', random_state=42)
regressor.fit(X_train, y_train)
#score
score = regressor.score(X_train, y_train)
print('Score: ', score)
print('Accuracy: ' + str(score*100) + '%')
print("MAE",np.sqrt( mean_absolute_error(y_train, regressor.predict(X_train) )))
print("MSE",np.sqrt( mean_squared_error(y_train, regressor.predict(X_train) )))

Score:  0.06692160611854685
Accuracy: 6.692160611854685%
MAE 121.48129709931928
MSE 18750.370153134074


C:\Users\Manan Bhatt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
